In [1]:
#Steel Plate Defect Prediction

In [4]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import roc_auc_score

In [5]:
path=r'/content/Challenge'

In [6]:
train_data = pd.read_csv(path + "/train.csv")
test_data = pd.read_csv(path + "/test.csv")

X = train_data.drop(columns=['id', 'Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults'])
y = train_data[['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']]
X = train_data.drop(columns=['id', 'Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults'])
y = train_data[['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [7]:
# 2. Model Definition
def create_model(input_shape, num_labels):
    model = models.Sequential([
        layers.Dense(64, activation='relu', input_shape=(input_shape,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_labels, activation='sigmoid')  # Sigmoid activation for multi-label classification
    ])
    return model

num_features = X_train_scaled.shape[1]
num_labels = y_train.shape[1]

model = create_model(num_features, num_labels)

# 3. Compilation
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Suitable for multi-label classification
              metrics=['accuracy'])

# Summary of the model
model.summary()

# 4. Training
history = model.fit(X_train_scaled, y_train,
                    epochs=100,
                    batch_size=32,
                    validation_data=(X_val_scaled, y_val))

# 5. Evaluation
val_predictions = model.predict(X_val_scaled)
auc_scores = []

for i, defect in enumerate(['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']):
    auc = roc_auc_score(y_val.iloc[:, i], val_predictions[:, i])
    auc_scores.append(auc)
    print(f"AUC for {defect}: {auc}")

avg_auc = sum(auc_scores) / len(auc_scores)
print(f"Average AUC: {avg_auc}")


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1792      
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 7)                 455       
                                                                 
Total params: 6407 (25.03 KB)
Trainable params: 6407 (25.03 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
481/481 [==============================] - 3s 3ms/step - loss: 0.2676 - accuracy: 0.5147 - val_loss: 0.2364 - val_accuracy: 0.5424
Epoch 2/100
481/481 [==============================] - 2s 4ms/step - loss: 0.2370 - accuracy: 0.5513 - val_loss: 0.2312 - val_accuracy: 0.5494


In [19]:
test_data_i = test_data.drop(columns=['id'])

In [20]:
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(test_data_i)

In [21]:
test_predictions = model.predict(X_test_scaled)

401/401 [==============================] - 1s 2ms/step


In [22]:
# Assuming `test_data` has an 'id' column and you've stored your predictions in `test_predictions`
submission_df = pd.DataFrame(test_predictions, columns=['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness',
                                                        'Bumps', 'Other_Faults'])
submission_df['id'] = test_data['id']

In [23]:
submission_df = submission_df[['id', 'Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']]

In [24]:
submission_df.to_csv(path + '/my_submission.csv', index=False)